## Creation of final database

It includes the creation of populist scores and information on the membership in governmnet and in the opposition.

In [107]:
# packages

import pandas as pd
import numpy as np
import os
import re


In [144]:
# os.chdir('C:/Users/...')

In [154]:
predicted = pd.read_csv("data/all_batches_predicted.csv")
labeled = pd.read_csv("data/labeled_new.csv")
leaders = pd.read_csv("data/leaders_new.csv")

### 3.1 Changing the structure of the datasets (idexes, missing values, dropping columns and filling the missing values)

In [155]:
labeled['combined_index'] = labeled['index'].astype(str) + '_' + labeled['paragraph_number'].astype(str)

In [156]:
predicted.drop(['predicted_label'], axis=1, inplace=True)
labeled.drop(['people-centrism', 'anti-elitism', 'populism'], axis=1, inplace=True)
labeled.rename(columns={'populism_full':'label_number'}, inplace=True)

In [157]:
full_data=pd.concat([predicted, labeled], ignore_index=True)

In [158]:
full_data['date'] = pd.to_datetime(full_data['date'], format='%d/%m/%Y')

In [159]:
full_data=leaders.merge(full_data, how='outer', on=['name', 'sejm'])

In [160]:
full_data['leader'] = full_data['leader'].fillna(0)

In [161]:
# if a person was a gov_memebr or in gov_lersership at any point in the term, it will have score 1
full_data['new_gov_member'] = full_data.groupby(['name', 'sejm'])['gov_member'].transform(lambda x: x.any())
full_data.loc[full_data['new_gov_member'] == True, 'gov_member'] = 1
full_data= full_data.drop('new_gov_member', axis=1)

# if a person was a minister at any point in the term it will have score 1
full_data['new_minister'] = full_data.groupby(['name', 'sejm'])['minister'].transform(lambda x: x.any())
full_data.loc[full_data['new_minister'] == True, 'minister'] = 1
full_data= full_data.drop('new_minister', axis=1)

# if a person was in a president office at any point in the term it will have score 1
full_data['new_president_office'] = full_data.groupby(['name', 'sejm'])['president_office'].transform(lambda x: x.any())
full_data.loc[full_data['new_president_office'] == True, 'president_office'] = 1
full_data= full_data.drop('new_president_office', axis=1)


full_data

,name,sejm,leader,legislator,debate,minister,gov_member,president_office,topic,link,speech,session,date,party_club_start,party_club_end,change,index,paragraph_number,combined_index,label_number
0,Adam Słomka,3,1.0,Adam Słomka,15.0,0.0,0.0,0.0,Pierwsze czytanie poselskich projektów uchwał:...,http://orka2.sejm.gov.pl/Debata3.nsf/5c30b337b...,Panie Marszałku! Wysoka Izbo! Rozliczenie prze...,15.0,1998-04-03,AWS,NZ,2.0,31911.0,1.0,31911_1,0.0
1,Adam Słomka,3,1.0,Adam Słomka,15.0,0.0,0.0,0.0,Pierwsze czytanie poselskich projektów uchwał:...,http://orka2.sejm.gov.pl/Debata3.nsf/5c30b337b...,Zatem zostajemy z tą dyskusją i debatą tylko w...,15.0,1998-04-03,AWS,NZ,2.0,31911.0,2.0,31911_2,0.0
2,Adam Słomka,3,1.0,Adam Słomka,15.0,0.0,0.0,0.0,Pierwsze czytanie poselskich projektów uchwał:...,http://orka2.sejm.gov.pl/Debata3.nsf/5c30b337b...,Przypomnimy - sąd karny w procesie morderców g...,15.0,1998-04-03,AWS,NZ,2.0,31911.0,3.0,31911_3,0.0
3,Adam Słomka,3,1.0,Adam Słomka,15.0,0.0,0.0,0.0,Pierwsze czytanie poselskich projektów uchwał:...,http://orka2.sejm.gov.pl/Debata3.nsf/5c30b337b...,"Niestety, tak się nie dzieje. Stąd niezbędne s...",15.0,1998-04-03,AWS,NZ,2.0,31911.0,4.0,31911_4,0.0
4,Adam Słomka,3,1.0,Adam Słomka,15.0,0.0,0.0,0.0,Pierwsze czytanie poselskich projektów uchwał:...,http://orka2.sejm.gov.pl/Debata3.nsf/5c30b337b...,miałem przyjemność proponować ustawę desowiety...,15.0,1998-04-03,AWS,NZ,2.0,31911.0,5.0,31911_5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012742,Tomasz Szczegielniak,8,0.0,Podsekretarz Stanu w Kancelarii Prezesa Rady M...,9.0,0.0,1.0,0.0,"Sprawozdanie Komisji Regulaminowej, Spraw Pose...",http://www.sejm.gov.pl/Sejm8.nsf/wypowiedz.xsp...,"Jeszcze nie wracamy, bo ustawa nie została uch...",86.0,2019-09-11,NaN,NaN,NaN,254021.0,3.0,254021_3,0.0
1012743,Tomasz Szczegielniak,8,0.0,Podsekretarz Stanu w Kancelarii Prezesa Rady M...,9.0,0.0,1.0,0.0,"Sprawozdanie Komisji Regulaminowej, Spraw Pose...",http://www.sejm.gov.pl/Sejm8.nsf/wypowiedz.xsp...,Oświadczenie majątkowe mają obowiązek złożyć o...,86.0,2019-09-11,NaN,NaN,NaN,254021.0,4.0,254021_4,0.0
1012744,Anna Dalkowska,8,0.0,Podsekretarz Stanu w Ministerstwie Sprawiedliw...,34.0,0.0,1.0,0.0,Pierwsze czytanie obywatelskiego projektu usta...,http://www.sejm.gov.pl/Sejm8.nsf/wypowiedz.xsp...,Dziękuję bardzo za udzielenie głosu. Panie Mar...,86.0,2019-10-15,NaN,NaN,NaN,254048.0,1.0,254048_1,0.0
1012745,Anna Dalkowska,8,0.0,Podsekretarz Stanu w Ministerstwie Sprawiedliw...,34.0,0.0,1.0,0.0,Pierwsze czytanie obywatelskiego projektu usta...,http://www.sejm.gov.pl/Sejm8.nsf/wypowiedz.xsp...,"Dlatego też, mając na względzie tę okoliczność...",86.0,2019-10-15,NaN,NaN,NaN,254048.0,2.0,254048_2,0.0


In [162]:
full_data.sort_values(['date'])

,name,sejm,leader,legislator,debate,minister,gov_member,president_office,topic,link,speech,session,date,party_club_start,party_club_end,change,index,paragraph_number,combined_index,label_number
53099,Ryszard Bugaj,2,1.0,Ryszard Bugaj,3.0,0.0,0.0,0.0,Wybór wicemarszałków Sejmu.,http://orka2.sejm.gov.pl/Debata2.nsf/5c30b337b...,Na dowód tego chciałbym przytoczyć pewien drob...,1.0,1993-10-14,UP,UP,0.0,9.0,4.0,9_4,0.0
65691,Mirosław Czech,2,0.0,Mirosław Czech,3.0,0.0,0.0,0.0,Wybór wicemarszałków Sejmu.,http://orka2.sejm.gov.pl/Debata2.nsf/5c30b337b...,Arkuszewski Piotr Aszyk Andrzej Aumiller Andrz...,1.0,1993-10-14,UD,UW,0.0,8.0,2.0,8_2,0.0
65692,Mirosław Czech,2,0.0,Mirosław Czech,3.0,0.0,0.0,0.0,Wybór wicemarszałków Sejmu.,http://orka2.sejm.gov.pl/Debata2.nsf/5c30b337b...,Stanisław Bartoszek Zygmunt Bartoszewicz Krzys...,1.0,1993-10-14,UD,UW,0.0,8.0,3.0,8_3,0.0
65693,Mirosław Czech,2,0.0,Mirosław Czech,3.0,0.0,0.0,0.0,Wybór wicemarszałków Sejmu.,http://orka2.sejm.gov.pl/Debata2.nsf/5c30b337b...,i Barbara Blida Wojciech Błasiak Lidia Błądek ...,1.0,1993-10-14,UD,UW,0.0,8.0,4.0,8_4,0.0
65694,Mirosław Czech,2,0.0,Mirosław Czech,3.0,0.0,0.0,0.0,Wybór wicemarszałków Sejmu.,http://orka2.sejm.gov.pl/Debata2.nsf/5c30b337b...,igniew Bomba Ryszard Bondyra Jan Borkowski Woj...,1.0,1993-10-14,UD,UW,0.0,8.0,5.0,8_5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60830,Zbigniew Puchajda,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60959,Zbigniew Witaszek,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60960,Zdzisław Jankowski,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60961,Zenon Tyma,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
full_data['party_club_start'] = full_data['party_club_start'].fillna('none')
full_data['party_club_end'] = full_data['party_club_end'].fillna('none')

In [164]:
full_data = full_data[(full_data['party_club_end'] != 'none') & (full_data['party_club_start'] != 'none')]

### 3.2 Creating the populist scores

In [165]:
#1997 before
start_date = '1997-03-21'
end_date = '1997-09-21'
data_range = full_data.loc[(full_data['date'] >= start_date) & (full_data['date'] <= end_date), :]
avg_score = data_range.groupby(['name', 'sejm'])['label_number'].mean()
df_before_97 = pd.DataFrame(avg_score.reset_index()).rename(columns={'label_number': 'before_97', 'sejm': 'sejm'})
df_before_97=pd.melt(df_before_97, id_vars=['name', 'sejm'], value_vars=['before_97'])


#1997 after
start_date_2 = '1997-09-21'
end_date_2 = '1998-03-21'
data_range_2 = full_data.loc[(full_data['date'] >= start_date_2) & (full_data['date'] <= end_date_2), :]
avg_score_2 = data_range_2.groupby(['name', 'sejm'])['label_number'].mean()
df_after_97 = pd.DataFrame(avg_score_2.reset_index()).rename(columns={'label_number': 'after_97', 'sejm': 'sejm'})
df_after_97=pd.melt(df_after_97, id_vars=['name', 'sejm'], value_vars=['after_97'])

In [166]:
#2001 before
start_date = '2001-03-23'
end_date = '2001-09-23'
data_range = full_data.loc[(full_data['date'] >= start_date) & (full_data['date'] <= end_date), :]
avg_score = data_range.groupby(['name', 'sejm'])['label_number'].mean()
df_before_01 = pd.DataFrame(avg_score.reset_index()).rename(columns={'label_number': 'before_01', 'sejm': 'sejm'})
df_before_01=pd.melt(df_before_01, id_vars=['name', 'sejm'], value_vars=['before_01'])

#2001 after
start_date_2 = '2001-09-23'
end_date_2 = '2002-03-23'
data_range_2 = full_data.loc[(full_data['date'] >= start_date_2) & (full_data['date'] <= end_date_2), :]
avg_score_2 = data_range_2.groupby(['name', 'sejm'])['label_number'].mean()
df_after_01 = pd.DataFrame(avg_score_2.reset_index()).rename(columns={'label_number': 'after_01', 'sejm': 'sejm'})
df_after_01=pd.melt(df_after_01, id_vars=['name', 'sejm'], value_vars=['after_01'])

In [167]:
#2005 before
start_date = '2005-03-25'
end_date = '2005-09-25'
data_range = full_data.loc[(full_data['date'] >= start_date) & (full_data['date'] <= end_date), :]
avg_score = data_range.groupby(['name', 'sejm'])['label_number'].mean()
df_before_05 = pd.DataFrame(avg_score.reset_index()).rename(columns={'label_number': 'before_05', 'sejm': 'sejm'})
df_before_05=pd.melt(df_before_05, id_vars=['name', 'sejm'], value_vars=['before_05'])

#2005 after
start_date_2 = '2005-09-25'
end_date_2 = '2006-03-25'
data_range_2 = full_data.loc[(full_data['date'] >= start_date_2) & (full_data['date'] <= end_date_2), :]
avg_score_2 = data_range_2.groupby(['name', 'sejm'])['label_number'].mean()
df_after_05 = pd.DataFrame(avg_score_2.reset_index()).rename(columns={'label_number': 'after_05', 'sejm': 'sejm'})
df_after_05=pd.melt(df_after_05, id_vars=['name', 'sejm'], value_vars=['after_05'])

In [168]:
#2007 before
start_date = '2007-04-21'
end_date = '2007-10-21'
data_range = full_data.loc[(full_data['date'] >= start_date) & (full_data['date'] <= end_date), :]
avg_score = data_range.groupby(['name', 'sejm'])['label_number'].mean()
df_before_07 = pd.DataFrame(avg_score.reset_index()).rename(columns={'label_number': 'before_07', 'sejm': 'sejm'})
df_before_07=pd.melt(df_before_07, id_vars=['name', 'sejm'], value_vars=['before_07'])

#2007 after
start_date_2 = '2007-10-21'
end_date_2 = '2008-04-21'
data_range_2 = full_data.loc[(full_data['date'] >= start_date_2) & (full_data['date'] <= end_date_2), :]
avg_score_2 = data_range_2.groupby(['name', 'sejm'])['label_number'].mean()
df_after_07 = pd.DataFrame(avg_score_2.reset_index()).rename(columns={'label_number': 'after_07', 'sejm': 'sejm'})
df_after_07=pd.melt(df_after_07, id_vars=['name', 'sejm'], value_vars=['after_07'])



In [169]:
#2011 before
start_date = '2011-04-09'
end_date = '2011-10-09'
data_range = full_data.loc[(full_data['date'] >= start_date) & (full_data['date'] <= end_date), :]
avg_score = data_range.groupby(['name', 'sejm'])['label_number'].mean()
df_before_11 = pd.DataFrame(avg_score.reset_index()).rename(columns={'label_number': 'before_11', 'sejm': 'sejm'})
df_before_11=pd.melt(df_before_11, id_vars=['name', 'sejm'], value_vars=['before_11'])

#2011 after
start_date_2 = '2011-10-09'
end_date_2 = '2012-04-09'
data_range_2 = full_data.loc[(full_data['date'] >= start_date_2) & (full_data['date'] <= end_date_2), :]
avg_score_2 = data_range_2.groupby(['name', 'sejm'])['label_number'].mean()
df_after_11 = pd.DataFrame(avg_score_2.reset_index()).rename(columns={'label_number': 'after_11', 'sejm': 'sejm'})
df_after_11=pd.melt(df_after_11, id_vars=['name', 'sejm'], value_vars=['after_11'])

In [170]:
# 2015 before
start_date = '2015-04-25'
end_date = '2015-10-25'
data_range = full_data.loc[(full_data['date'] >= start_date) & (full_data['date'] <= end_date), :]
avg_score = data_range.groupby(['name', 'sejm'])['label_number'].mean()
df_before_15 = pd.DataFrame(avg_score.reset_index()).rename(columns={'label_number': 'before_15', 'sejm': 'sejm'})
df_before_15=pd.melt(df_before_15, id_vars=['name', 'sejm'], value_vars=['before_15'])

# 2015 after
start_date_2 = '2015-10-25'
end_date_2 = '2016-04-25'
data_range_2 = full_data.loc[(full_data['date'] >= start_date_2) & (full_data['date'] <= end_date_2), :]
avg_score_2 = data_range_2.groupby(['name', 'sejm'])['label_number'].mean()
df_after_15 = pd.DataFrame(avg_score_2.reset_index()).rename(columns={'label_number': 'after_15', 'sejm': 'sejm'})
df_after_15=pd.melt(df_after_15, id_vars=['name', 'sejm'], value_vars=['after_15'])


In [171]:
full_data.drop(['debate', 'topic', 'link','speech', 'session', 'date', 'index', 'paragraph_number', 'combined_index', 'label_number','legislator'], axis=1, inplace=True)
full_data = full_data.drop_duplicates()
full_data= full_data.reset_index(drop=True)

C:\Users\Anna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [172]:
new_df=pd.concat([df_before_97,df_after_97, df_before_01,df_after_01, df_before_07,df_after_07, df_before_11,df_after_11, df_before_15,df_after_15])
merged_df = new_df.merge(full_data, how='left', on=['name', 'sejm'])
merged_df.sort_values(['name','sejm'])

,name,sejm,variable,value,leader,minister,gov_member,president_office,party_club_start,party_club_end,change
1871,Adam Abramowicz,6,after_07,0.047619,0.0,0.0,0.0,0.0,PIS,PIS,0.0
2281,Adam Abramowicz,6,before_11,0.000000,0.0,0.0,0.0,0.0,PIS,PIS,0.0
2686,Adam Abramowicz,7,after_11,0.000000,0.0,0.0,0.0,0.0,PIS,PIS,0.0
3080,Adam Abramowicz,7,before_15,0.011905,0.0,0.0,0.0,0.0,PIS,PIS,0.0
3472,Adam Abramowicz,8,after_15,0.000000,0.0,0.0,0.0,0.0,PIS,PIS,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2280,Łukasz Zbonikowski,6,after_07,0.142857,0.0,0.0,0.0,0.0,PIS,PIS,0.0
2685,Łukasz Zbonikowski,6,before_11,0.064516,0.0,0.0,0.0,0.0,PIS,PIS,0.0
3079,Łukasz Zbonikowski,7,after_11,0.031250,0.0,0.0,0.0,0.0,PIS,PIS,0.0
3471,Łukasz Zbonikowski,7,before_15,0.000000,0.0,0.0,0.0,0.0,PIS,PIS,0.0


In [173]:
merged_df["index"] = merged_df["variable"].str.replace("(before|after)_\d+", "\\1")
merged_df = merged_df.drop("variable", axis=1)
wide_data = merged_df.pivot(index=["name", "sejm", "leader", "minister","gov_member","president_office","party_club_start","party_club_end","change"], columns="index", values="value")
wide_data = wide_data.reset_index()

C:\Users\Anna\AppData\Local\Temp\ipykernel_6372\1603853838.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_df["index"] = merged_df["variable"].str.replace("(before|after)_\d+", "\\1")


In [174]:
wide_data

index,name,sejm,leader,minister,gov_member,president_office,party_club_start,party_club_end,change,after,before
0,Adam Abramowicz,6,0.0,0.0,0.0,0.0,PIS,PIS,0.0,0.047619,0.000000
1,Adam Abramowicz,7,0.0,0.0,0.0,0.0,PIS,PIS,0.0,0.000000,0.011905
2,Adam Abramowicz,8,0.0,0.0,0.0,0.0,PIS,PIS,0.0,0.000000,NaN
3,Adam Andruszkiewicz,8,0.0,0.0,0.0,0.0,Kukiz,NZ,2.0,0.000000,NaN
4,Adam Biela,3,0.0,0.0,0.0,0.0,AWS,AWS,0.0,0.090909,0.014706
...,...,...,...,...,...,...,...,...,...,...,...
2955,Łukasz Tusk,7,0.0,0.0,0.0,0.0,PO,PO,0.0,NaN,0.000000
2956,Łukasz Zbonikowski,5,0.0,0.0,0.0,0.0,PIS,PIS,0.0,NaN,0.000000
2957,Łukasz Zbonikowski,6,0.0,0.0,0.0,0.0,PIS,PIS,0.0,0.142857,0.064516
2958,Łukasz Zbonikowski,7,0.0,0.0,0.0,0.0,PIS,PIS,0.0,0.031250,0.000000


In [175]:
grouped = merged_df.groupby(['name', 'sejm']).apply(
    lambda x: x.iloc[1]['value'] - x.iloc[0]['value'] if len(x) > 1 else 0
).reset_index(name='diff')

grouped= grouped.reset_index(drop=True)


In [176]:
grouped_df = grouped.merge(full_data, how='left', on=['name', 'sejm'])
grouped_df.sort_values(['name','sejm'])

# diff= BEFORE-AFTER eg. Just before elections in 2011 - just after the elections in 2007

,name,sejm,diff,leader,minister,gov_member,president_office,party_club_start,party_club_end,change
0,Adam Abramowicz,6,-0.047619,0.0,0.0,0.0,0.0,PIS,PIS,0.0
1,Adam Abramowicz,7,0.011905,0.0,0.0,0.0,0.0,PIS,PIS,0.0
2,Adam Abramowicz,8,0.000000,0.0,0.0,0.0,0.0,PIS,PIS,0.0
3,Adam Andruszkiewicz,8,0.000000,0.0,0.0,0.0,0.0,Kukiz,NZ,2.0
4,Adam Biela,3,-0.076203,0.0,0.0,0.0,0.0,AWS,AWS,0.0
...,...,...,...,...,...,...,...,...,...,...
2955,Łukasz Tusk,7,0.000000,0.0,0.0,0.0,0.0,PO,PO,0.0
2956,Łukasz Zbonikowski,5,0.000000,0.0,0.0,0.0,0.0,PIS,PIS,0.0
2957,Łukasz Zbonikowski,6,-0.078341,0.0,0.0,0.0,0.0,PIS,PIS,0.0
2958,Łukasz Zbonikowski,7,-0.031250,0.0,0.0,0.0,0.0,PIS,PIS,0.0


### 3.3 Adding information on the membership in government and in opposition

In [177]:
# information on the membership in govermnet and in opposition
# term 2
gov_2=['SLD', 'PSL']	
opp_2=['UD','UW', 'AWS', 'BBWR', 'KPN', 'PPS']

# term 3
gov_3=['AWS', 'UW']
opp_3=['PSL', 'SLD', 'ROP']	

# term 4
gov_4=['SLD-UP', 'PSL', 'SLD']
opp_4=['PSL', 'PO', 'PIS', 'LPR', 'SKL', 'RKN', 'PP', 'ROP', 'Dom Ojczysty']

# term 5
gov_5=['PIS', 'Samoobrona', 'LPR']
opp_5=['SLD', 'PSL', 'PO']

# term 6
gov_6= ['PO', 'PSL']
opp_6=['SLD', 'PIS', 'SDPL', 'PJN']

# term 7
gov_7=['PO', 'PSL']
opp_7=['SLD', 'PIS', 'ZP']

#term 8
gov_8=['PIS']
opp_8=['SLD', 'PSL-KP', 'PO-KO', 'Teraz', 'Nowoczesna']


In [178]:

grouped_df['gov'] = 0
mask_start = grouped_df['party_club_start'].isin(gov_2) & (grouped_df['sejm'] == 2)
mask_end = grouped_df['party_club_end'].isin(gov_2) & (grouped_df['sejm'] == 2)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(gov_3) & (grouped_df['sejm'] == 3)
mask_end = grouped_df['party_club_end'].isin(gov_3) & (grouped_df['sejm'] == 3)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(gov_4) & (grouped_df['sejm'] == 4)
mask_end = grouped_df['party_club_end'].isin(gov_4) & (grouped_df['sejm'] == 4)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(gov_5) & (grouped_df['sejm'] == 5)
mask_end = grouped_df['party_club_end'].isin(gov_5) & (grouped_df['sejm'] == 5)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(gov_6) & (grouped_df['sejm'] == 6)
mask_end = grouped_df['party_club_end'].isin(gov_6) & (grouped_df['sejm'] == 6)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(gov_7) & (grouped_df['sejm'] == 7)
mask_end = grouped_df['party_club_end'].isin(gov_7) & (grouped_df['sejm'] == 7)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(gov_8) & (grouped_df['sejm'] == 8)
mask_end = grouped_df['party_club_end'].isin(gov_8) & (grouped_df['sejm'] == 8)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

In [179]:

grouped_df['opp'] = 0
mask_start = grouped_df['party_club_start'].isin(opp_2) & (grouped_df['sejm'] == 2)
mask_end = grouped_df['party_club_end'].isin(opp_2) & (grouped_df['sejm'] == 2)
grouped_df.loc[mask_start | mask_end, 'opp'] = 1

mask_start = grouped_df['party_club_start'].isin(opp_3) & (grouped_df['sejm'] == 3)
mask_end = grouped_df['party_club_end'].isin(opp_3) & (grouped_df['sejm'] == 3)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(opp_4) & (grouped_df['sejm'] == 4)
mask_end = grouped_df['party_club_end'].isin(opp_4) & (grouped_df['sejm'] == 4)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(opp_5) & (grouped_df['sejm'] == 5)
mask_end = grouped_df['party_club_end'].isin(opp_5) & (grouped_df['sejm'] == 5)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(opp_6) & (grouped_df['sejm'] == 6)
mask_end = grouped_df['party_club_end'].isin(opp_6) & (grouped_df['sejm'] == 6)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(opp_7) & (grouped_df['sejm'] == 7)
mask_end = grouped_df['party_club_end'].isin(opp_7) & (grouped_df['sejm'] == 7)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

mask_start = grouped_df['party_club_start'].isin(opp_8) & (grouped_df['sejm'] == 8)
mask_end = grouped_df['party_club_end'].isin(opp_8) & (grouped_df['sejm'] == 8)
grouped_df.loc[mask_start | mask_end, 'gov'] = 1

In [180]:
grouped_df

,name,sejm,diff,leader,minister,gov_member,president_office,party_club_start,party_club_end,change,gov,opp
0,Adam Abramowicz,6,-0.047619,0.0,0.0,0.0,0.0,PIS,PIS,0.0,1,0
1,Adam Abramowicz,7,0.011905,0.0,0.0,0.0,0.0,PIS,PIS,0.0,1,0
2,Adam Abramowicz,8,0.000000,0.0,0.0,0.0,0.0,PIS,PIS,0.0,1,0
3,Adam Andruszkiewicz,8,0.000000,0.0,0.0,0.0,0.0,Kukiz,NZ,2.0,0,0
4,Adam Biela,3,-0.076203,0.0,0.0,0.0,0.0,AWS,AWS,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2955,Łukasz Tusk,7,0.000000,0.0,0.0,0.0,0.0,PO,PO,0.0,1,0
2956,Łukasz Zbonikowski,5,0.000000,0.0,0.0,0.0,0.0,PIS,PIS,0.0,1,0
2957,Łukasz Zbonikowski,6,-0.078341,0.0,0.0,0.0,0.0,PIS,PIS,0.0,1,0
2958,Łukasz Zbonikowski,7,-0.031250,0.0,0.0,0.0,0.0,PIS,PIS,0.0,1,0


In [74]:
#grouped_df = grouped_df.drop_duplicates()

In [182]:
grouped_df.to_csv('data/scores.csv', encoding='utf-8', index=False)